## 取得資料

In [1]:
import requests
import pandas as pd

url = "http://192.168.31.130:32327/games_reviews/_search?scroll=1m"  # 使用 Scroll，scroll=1m 表示 Scroll 保持活躍1分鐘
headers = {"Content-Type": "application/json"}

# 初始化 Scroll 查詢，指定要返回的欄位
data = {
    "_source": [
        "steamId",  # 正確添加 steamId 欄位
        "twitch_data",  # 查詢 twitch_data
        "Online_by_month",  # 查詢 Online_by_month
    ],
    "query": {"match_all": {}},  # 匹配所有文件
    "size": 1000,  # 每次返回1000筆資料
}

response = requests.get(url, headers=headers, json=data)
response_json = response.json()

# 提取初始查詢的 hits 和 scroll_id
hits = response_json["hits"]["hits"]
scroll_id = response_json["_scroll_id"]

# 存儲資料的列表
data_list = [
    {
        "steamId": hit["_source"].get("steamId"),
        "twitch_data": hit["_source"].get("twitch_data"),
        "Online_by_month": hit["_source"].get("Online_by_month"),
    }
    for hit in hits
]

# 使用 Scroll API 繼續獲取剩餘的資料
while len(hits) > 0:
    scroll_url = "http://192.168.31.130:32327/_search/scroll"
    scroll_data = {
        "scroll": "1m",  # Scroll 保持活躍1分鐘
        "scroll_id": scroll_id,  # 使用前次查詢返回的 scroll_id 繼續查詢
    }

    scroll_response = requests.get(scroll_url, headers=headers, json=scroll_data)
    scroll_response_json = scroll_response.json()

    hits = scroll_response_json["hits"]["hits"]
    scroll_id = scroll_response_json["_scroll_id"]

    # 將新獲取的 hits 資料追加到列表中
    data_list.extend(
        [
            {
                "steamId": hit["_source"].get("steamId"),
                "twitch_data": hit["_source"].get("twitch_data"),
                "Online_by_month": hit["_source"].get("Online_by_month"),
            }
            for hit in hits
        ]
    )

# 將提取的資料轉換為 DataFrame
df = pd.DataFrame(data_list)

# 顯示資料
df

,steamId,twitch_data,Online_by_month
0,240,"[Oct 2024, 6, -18, -75%, 48, 2, -, -, 7, 351, ...","[[Last 30, 8436.63, -1847.2, -17.96%, 13793], ..."
1,80,"[Oct 2024, 1, -1, -50%, 5, 1, -, -, 2, 14, Sep...","[[Last 30, 306.69, -2.4, -0.78%, 537], [August..."
2,262280,"[Sep 2024, 1, -, -, 8, 1, -, -, 2, 88, Aug 202...","[[Last 30, 18.74, -1.5, -7.29%, 52], [August 2..."
3,1179680,"[Oct 2024, 17, -13, -43.3%, 74, 1, -, -, 3, 52...","[[Last 30, 278.63, +115.6, +70.91%, 506], [Aug..."
4,688130,"[Oct 2024, 92, +6, +7%, 249, 5, +1, +25%, 16, ...","[[Last 30, 48.54, -6.7, -12.14%, 100], [August..."
...,...,...,...
10331,2195410,"[Oct 2024, 1, -36, -97.3%, 2, 1, -, -, 1, 3, S...","[[Last 30, 22.17, +5.5, +32.91%, 82], [August ..."
10332,2293680,"[Oct 2024, 1, -61, -98.4%, 2, 1, -, -, 1, 1, S...","[[Last 30, 20.69, -6.3, -23.45%, 61], [August ..."
10333,262100,"[Mar 2021, 1, -, -, 1, 1, -, -, 1, 1]","[[Last 30, 0.00, -0.0, -100.00%, 0], [Septembe..."
10334,1277870,"[May 2024, 201, +200, +20.0K%, 209, 1, -, -, 1...",[]


## online_by_month

### 處理 online_by_month 的原始資料，並計算每個遊戲ID各自擁有的資料數量 (數量顯示在lens)。

In [2]:
# 計算每個steamID的對應的Online_by_month，一共有幾筆時間序列，計算結果顯示在lens

online_df_try = df[["steamId", "Online_by_month"]]  # 從大表取出所需表格

online_len = []  # 創建空集合

for column in online_df_try["Online_by_month"]:

    if column is not None:  # 確認column不為空值

        len_each = len(column)  # 計算 lens 數量，1個lens就代表1個時間點序列
        online_len.append(len_each)  # 將計算結果append回空集合

    else:
        online_len.append(0)  # 若 column 為空值則直接append一個0

online_df_try["lens"] = pd.DataFrame(online_len)  # 轉成dataframe

online_df_try

,steamId,Online_by_month,lens
0,240,"[[Last 30, 8436.63, -1847.2, -17.96%, 13793], ...",147
1,80,"[[Last 30, 306.69, -2.4, -0.78%, 537], [August...",147
2,262280,"[[Last 30, 18.74, -1.5, -7.29%, 52], [August 2...",114
3,1179680,"[[Last 30, 278.63, +115.6, +70.91%, 506], [Aug...",50
4,688130,"[[Last 30, 48.54, -6.7, -12.14%, 100], [August...",67
...,...,...,...
10331,2195410,"[[Last 30, 22.17, +5.5, +32.91%, 82], [August ...",4
10332,2293680,"[[Last 30, 20.69, -6.3, -23.45%, 61], [August ...",3
10333,262100,"[[Last 30, 0.00, -0.0, -100.00%, 0], [Septembe...",37
10334,1277870,[],0


### 為了將時間序列資料展開後仍可對應遊戲ID，因此先以Lens計算出每個遊戲ID應更改之數量。

In [3]:
# 建立空集合
id = []

for x in range(online_df_try.shape[0]):
    if online_df_try["lens"].loc[x] != 0:  # 確認lens的column為不為0

        x_ = (
            online_df_try["steamId"].loc[x]
            * online_df_try["lens"].loc[x]  # 得出一個遊戲id會有多少筆時間序列的資料
        )

        x_split = ",".join(
            [
                x_[i : i + len(online_df_try["steamId"].loc[x])]
                for i in range(
                    0, len(x_), len(online_df_try["steamId"].loc[x]) # 將資料根據遊戲id的長度進行分割
                )  
            ]
        )

        id.append(x_split)  # append回id集合
    else:
        id.append(online_df_try["steamId"].loc[x])  # 若lens為0，則直接append 1個遊戲id

df_id = pd.DataFrame(id, columns=["steamID"])  # 將資料轉換成 dataframe 格式


# 此時每個row的資料尚未展開，因此先將資料分割，在做展開的動作
df_id["steamID"] = df_id["steamID"].str.split(",")
df_id_new = df_id["steamID"].explode().reset_index(drop=True)

z = pd.DataFrame(df_id_new)  # 最後得出最終的資料dataframe

z

,steamID
0,240
1,240
2,240
3,240
4,240
...,...
183815,262100
183816,262100
183817,262100
183818,1277870


### 將整理好的Online_by_month資料展開，並存成dataframe，同時將空值轉換成0。

In [4]:
# 建立空集合
online_data = []

for column in df["Online_by_month"]:
    if column:  # 检查是否为非空列表
        for index in column:
            online_data.append(index)
    else:
        online_data.append("-")

df_grouped = pd.DataFrame(
    online_data,
    columns=['Date', 'Avg.Players', 'Gain', '% Gain', 'Peak Players'],
)

df_filled_grouped = df_grouped.fillna(0)
df_filled_grouped = df_filled_grouped.replace('-', 0)

df_filled_grouped

,Date,Avg.Players,Gain,% Gain,Peak Players
0,Last 30,8436.63,-1847.2,-17.96%,13793
1,August 2024,10283.87,-21.08,-0.20%,15935
2,July 2024,10304.96,1833.97,+21.65%,17981
3,June 2024,8470.98,-710.44,-7.74%,13920
4,May 2024,9181.42,1224.42,+15.39%,12756
...,...,...,...,...,...
183815,June 2016,0.49,0.13,+37.39%,26
183816,May 2016,0.35,-1.82,-83.72%,5
183817,April 2016,2.18,0,0,14
183818,0,0,0,0,0


### 將整理好的數據和擴展好的steamID做結合，並重新排序column順序

In [5]:
# 新增一個column，將擴展好的steamId做結合

df_filled_grouped["steamId"] = z["steamID"] 

In [6]:
# 整理新的column順序

new_oder = ['steamId', 'Date', 'Avg.Players', 'Gain', '% Gain', 'Peak Players']

df_filled_grouped = df_filled_grouped[new_oder]

df_filled_grouped

,steamId,Date,Avg.Players,Gain,% Gain,Peak Players
0,240,Last 30,8436.63,-1847.2,-17.96%,13793
1,240,August 2024,10283.87,-21.08,-0.20%,15935
2,240,July 2024,10304.96,1833.97,+21.65%,17981
3,240,June 2024,8470.98,-710.44,-7.74%,13920
4,240,May 2024,9181.42,1224.42,+15.39%,12756
...,...,...,...,...,...,...
183815,262100,June 2016,0.49,0.13,+37.39%,26
183816,262100,May 2016,0.35,-1.82,-83.72%,5
183817,262100,April 2016,2.18,0,0,14
183818,1277870,0,0,0,0,0


## twitch_data

### 處理 twitch_data 的原始資料，並計算每個遊戲ID各自擁有的資料數量 (數量顯示在lens)。

In [7]:
# 計算每個steamID的對應的twitch_data，一共有幾筆時間序列，計算結果顯示在lens

twitch_df_try = df[["steamId", "twitch_data"]]  # 從大表取出所需表格

twitch_len = []  # 創建空集合

for column in twitch_df_try["twitch_data"]:

    if column is not None:  # 確認column不為空值

        len_each = len(column)  # 計算 lens 數量，1個lens就代表1個時間點序列
        twitch_len.append(len_each)  # 將計算結果append回空集合

    else:
        twitch_len.append(0)  # 若 column 為空值則直接append一個0

twitch_df_try["lens"] = pd.DataFrame(twitch_len)  # 轉成dataframe

twitch_df_try.head(20)

,steamId,twitch_data,lens
0,240,"[Oct 2024, 6, -18, -75%, 48, 2, -, -, 7, 351, ...",950
1,80,"[Oct 2024, 1, -1, -50%, 5, 1, -, -, 2, 14, Sep...",950
2,262280,"[Sep 2024, 1, -, -, 8, 1, -, -, 2, 88, Aug 202...",940
3,1179680,"[Oct 2024, 17, -13, -43.3%, 74, 1, -, -, 3, 52...",950
4,688130,"[Oct 2024, 92, +6, +7%, 249, 5, +1, +25%, 16, ...",690
5,1966240,[],0
6,1835210,[],0
7,983260,"[May 2024, 0, -, -, 0, 1, -, -, 1, 0]",10
8,2228310,[],0
9,2097160,"[May 2024, 3, +1, +50%, 3, 1, -, -, 1, 3, Feb ...",90


### 將處理好的twitch_data做展開。

In [8]:
twitch_data = []


for column in twitch_df_try["twitch_data"]:
    if column is not None or []:
        for index in column:
            twitch_data.append(index)
    
    else:
        twitch_data.append(0)

twitch_data

['Oct 2024',
 '6',
 '-18',
 '-75%',
 '48',
 '2',
 '-',
 '-',
 '7',
 '351',
 'Sep 2024',
 '24',
 '+5',
 '+26.3%',
 '32,229',
 '2',
 '-1',
 '-33.3%',
 '12',
 '15.0K',
 'Aug 2024',
 '19',
 '-80',
 '-80.8%',
 '21,696',
 '3',
 '-',
 '-',
 '13',
 '11.9K',
 'Jul 2024',
 '99',
 '+77',
 '+350%',
 '14,339',
 '3',
 '+1',
 '+50%',
 '12',
 '66.7K',
 'Jun 2024',
 '22',
 '+11',
 '+100%',
 '3,763',
 '2',
 '-',
 '-',
 '11',
 '14.8K',
 'May 2024',
 '11',
 '-5',
 '-31.3%',
 '4,223',
 '2',
 '-',
 '-',
 '10',
 '7.3K',
 'Apr 2024',
 '16',
 '-12',
 '-42.9%',
 '13,771',
 '2',
 '-1',
 '-33.3%',
 '9',
 '11.1K',
 'Mar 2024',
 '28',
 '+15',
 '+115%',
 '4,466',
 '3',
 '-',
 '-',
 '14',
 '18.3K',
 'Feb 2024',
 '13',
 '-1',
 '-7.1%',
 '1,644',
 '3',
 '-',
 '-',
 '11',
 '8.3K',
 'Jan 2024',
 '14',
 '-4',
 '-22.2%',
 '4,983',
 '3',
 '-',
 '-',
 '13',
 '8.8K',
 'Dec 2023',
 '18',
 '+7',
 '+63.6%',
 '3,597',
 '3',
 '+1',
 '+50%',
 '12',
 '11.9K',
 'Nov 2023',
 '11',
 '-',
 '-',
 '855',
 '2',
 '-1',
 '-33.3%',
 '11',
 '7

### 展開後的資料預設10個字元為一組，將資料重新組合成dataframe格式。

In [9]:
grouped_data = []


# 每 10 行分為一組
for i in range(0, len(twitch_data), 10):
    grouped_data.append(twitch_data[i : i + 10])

# 將數據轉換為 DataFrame 並設置列名
df_grouped = pd.DataFrame(
    grouped_data,
    columns=[
        "Date",
        "Avg Viewers",
        "Gain",
        "% Gain",
        "Peak Viewers",
        "Avg Streams",
        "Gain",
        "% Gain",
        "Peak Streams",
        "Hours Watched",
    ],
)

df_grouped.head(450)

,Date,Avg Viewers,Gain,% Gain,Peak Viewers,Avg Streams,Gain,% Gain,Peak Streams,Hours Watched
0,Oct 2024,6,-18,-75%,48,2,-,-,7,351
1,Sep 2024,24,+5,+26.3%,"32,229",2,-1,-33.3%,12,15.0K
2,Aug 2024,19,-80,-80.8%,"21,696",3,-,-,13,11.9K
3,Jul 2024,99,+77,+350%,"14,339",3,+1,+50%,12,66.7K
4,Jun 2024,22,+11,+100%,"3,763",2,-,-,11,14.8K
...,...,...,...,...,...,...,...,...,...,...
445,Apr 2019,59,"-2,842",-98%,"2,694",2,-10,-83.3%,10,30.6K
446,Mar 2019,"2,901",+798,+37.9%,"56,704",12,+11,+1.1K%,55,2.16M
447,Feb 2019,"2,103",-,-,"9,788",1,-,-,10,303K
448,May 2024,0,-,-,0,1,-,-,1,0


In [10]:
# df_grouped.columns = ['Date', 'Avg Viewers', 'Viewers Gain', 'Viewers % Gain', 'Peak Viewers', 'Avg Streams',
       # 'Streams Gain', 'Streams % Gain', 'Peak Streams', 'Hours Watched']